In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
#COURSE
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [6]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'ID_Course': 1,
  'CourseCode': 'BC-DATAVIZ',
  'CourseName': 'Data Visualization and Analytics',
  'CreditHours': 12,
  'BootCampCourse': 1},
 {'ID_Course': 2,
  'CourseCode': 'BC-WEBDEV',
  'CourseName': 'Full Stack Web Development',
  'CreditHours': 12,
  'BootCampCourse': 1},
 {'ID_Course': 3,
  'CourseCode': 'BC-UIUX',
  'CourseName': 'User Interface/User Experience',
  'CreditHours': 12,
  'BootCampCourse': 1},
 {'ID_Course': 4,
  'CourseCode': 'CIS-349',
  'CourseName': 'Introduction to Databases',
  'CreditHours': 5,
  'BootCampCourse': 0},
 {'ID_Course': 5,
  'CourseCode': 'CIS-405',
  'CourseName': 'Database Programming',
  'CreditHours': 5,
  'BootCampCourse': 0},
 {'ID_Course': 6,
  'CourseCode': 'CIS-438',
  'CourseName': 'Database Administration',
  'CreditHours': 5,
  'BootCampCourse': 0},
 {'ID_Course': 7,
  'CourseCode': 'CIS-430',
  'CourseName': 'Business Systems Programming I',
  'CreditHours': 5,
  'BootCampCourse': 0},
 {'ID_Course': 8,
  'CourseCode': 'CIS-435'

In [7]:
for rec in course_data_records:

    record = {
        'ID_Course__c': rec['ID_Course'],
        'CourseCode__c': rec['CourseCode'],
        'CourseName__c': rec['CourseName'],
        'CreditHours__c': rec['CreditHours'],
        'BootCampCourse__c': rec['BootCampCourse'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)

Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Course__C/. Response content: [{'message': 'Cannot deserialize instance of double from VALUE_STRING value BC-DATAVIZ or request may be missing a required field at [line:1, column:19]', 'errorCode': 'JSON_PARSER_ERROR'}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Course__C/. Response content: [{'message': 'Cannot deserialize instance of double from VALUE_STRING value BC-WEBDEV or request may be missing a required field at [line:1, column:19]', 'errorCode': 'JSON_PARSER_ERROR'}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Course__C/. Response content: [{'message': 'Cannot deserialize instance of double from VALUE_STRING value BC-UIUX or request may be missing a required field at [line:1, column:19]', 'errorCode': 'JSON_PARSER_ERROR'}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Course__C/. Response content: [{'message

In [8]:
course_lookup_list = []
data = sf.query_all_iter("SELECT ID_Course__c, CourseName__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course': row['ID_Course__c'],
        'CourseName': row['CourseName__c'],
        'CourseSF': row['Name']
    }
    course_lookup_list.append(rec)

SalesforceMalformedRequest: Malformed request https://na111.salesforce.com/services/data/v42.0/query/?q=SELECT+ID_Course__c%2C+CourseName__c%2C+Name+FROM+Course__c. Response content: [{'message': "\nSELECT ID_Course__c, CourseName__c, Name FROM Course__c\n                     ^\nERROR at Row:1:Column:22\nNo such column 'CourseName__c' on entity 'Course__c'. If you are attempting to use a custom field, be sure to append the '__c' after the custom field name. Please reference your WSDL or the describe call for the appropriate names.", 'errorCode': 'INVALID_FIELD'}]

In [9]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

""


In [10]:
#CLASS
query = '''
    SELECT
        c.*
        
    FROM
        class c
        
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [11]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
class_data_df.head()

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [12]:
class_data_df['StartDate'] = pd.to_datetime(class_data_df['StartDate']).dt.date
class_data_df['EndDate'] = pd.to_datetime(class_data_df['EndDate']).dt.date
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [13]:
for index, row in class_data_df.iterrows():
        record = {
        'ID_Class__c': row['ID_Class'],
        'ID_Course__c' : row['ID_Course'],
        'Section__c': row['Section'],
        'StartDate__c': str(row['StartDate']),
        'EndDate__c': str(row['EndDate']),
        'Course_Name__c': row['CourseSF']
    }
    
        try:
            sf.Class__c.create(record)
        except Exception as e:
            print(e)

KeyError: 'CourseSF'